### Cleaning of tweets

In [20]:
import pandas as pd
import numpy as np

In [32]:
df=pd.read_csv('Covid19/March.csv')

In [33]:
import re #regular expressions
import string 
from nltk.corpus import stopwords
from nltk import word_tokenize

In [34]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

In [35]:
#replace tabs and spaces
df['Text']= df['Text'].replace({'\n':" ","\t":" "})
df['Text']

0        In the war against the global epidemic #corona...
1        Dear liberals, Please do not pray for Corona v...
2        Sir there is not treatment available for coron...
3        Replying to #WHO "Coronavirus confirmed as pan...
4        #jharkhand @HemantSorenJMM @WeAreRanchi @WeAre...
                               ...                        
42313    The members of Tableegi jamaat in Sopore hosip...
42314    Iranian state TV said Saturday the new coronav...
42315    #Breaking | 5 more cases of Coronavirus have b...
42316    CORONAVIRUS I'm not scared of the virus, I'm j...
42317    Asalamu Alikum Good decision makers can make i...
Name: Text, Length: 42318, dtype: object

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aamir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aamir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [36]:
import dask.dataframe as dsk
import multiprocessing as mp
from dask.multiprocessing import get
ddf = dsk.from_pandas(df,npartitions=4*mp.cpu_count())

In [37]:
df['Text1'] = df['Text'].apply(clean_tweets) #preprocessing using user defined functions
df.columns

Index(['Unnamed: 0', 'Place', 'Query', 'Datetime', 'Text', 'retweets',
       'favourites', 'hashtags', 'Text1'],
      dtype='object')

In [38]:
df['Text'] = df['Text1']
df = df.drop('Text1', axis = 1)

### Calculating subjectivity and Polarity

In [39]:
from textblob import TextBlob
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [40]:
df['Subjectivity']=df['Text'].apply(getSubjectivity)
df['Polarity']=df['Text'].apply(getPolarity)

In [41]:
df.columns

Index(['Unnamed: 0', 'Place', 'Query', 'Datetime', 'Text', 'retweets',
       'favourites', 'hashtags', 'Subjectivity', 'Polarity'],
      dtype='object')

In [42]:
df.head()

,Unnamed: 0,Place,Query,Datetime,Text,retweets,favourites,hashtags,Subjectivity,Polarity
0,0,Jharkhand,coronavirus,2020-03-29 16:23:31+00:00,In war global epidemic coronavirus I donated ₹...,1,9,#coronavirus #PMCARES,0.000000,0.000000
1,1,Jharkhand,coronavirus,2020-03-18 05:40:36+00:00,Dear liberals Please pray Corona virus cases g...,0,0,NaN,0.400000,-0.150000
2,2,Jharkhand,coronavirus,2020-03-19 08:09:44+00:00,Sir treatment available corona virus Jharkhand,0,0,NaN,0.400000,0.400000
3,3,Jharkhand,covid19,2020-03-11 18:05:59+00:00,Replying WHO `` Coronavirus confirmed pandemic...,0,2,#WHO #COVID19,0.597222,0.119444
4,4,Jharkhand,coronavirus,2020-03-26 08:26:04+00:00,jharkhand HemantSorenJMM WeAreRanchi WeAreDhan...,0,0,#jharkhand,0.225000,-0.058333


In [43]:
df.to_csv('NewMARCH.csv')

### Labelling tweets accoridng to sentiment

In [18]:
import pandas as pd

from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
nltk.download('punkt')


def emotion(tweet):
    
    def most_frequent(List): 
        return max(set(List), key = List.count) 
    
    
    word_tokens = word_tokenize(tweet)
    temp = []
    cleanedList = []
    i = 0
    for word in word_tokens:
        em = emot['emotion'].where(word == emot['word'])
        cleanedList = [x for x in em if str(x) != 'nan']

        if i == 0:
            if not cleanedList:
                continue
            else:
                temp = cleanedList
        else:
            if not cleanedList:
                continue
            else:
                temp.append(cleanedList[0])
        i += 1

   # print(f'temp : {temp}, len(temp) : {len(temp)}')
    if len(temp) == 0:
        #print(f'return: neutral')
        return 'neutral'
    
   # print(f'return: {most_frequent(temp)}')
    
    return most_frequent(temp)



df = pd.read_csv("NewJune.csv")
df = pd.DataFrame(df)

emot = pd.read_csv("emotions.csv")
emot = pd.DataFrame(emot)
emot.columns=['word', 'emotion']
df['Text']=df['Text'].apply(str)
df['sentiment'] = df['Text'].apply(emotion)
#print(df.sentiment.unique())

df.to_csv("june_sentiment.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aamir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df1=pd.read_csv('april_sentiment.csv')

In [15]:
df1['sentiment'].value_counts()

neutral     38207
joy          8155
surprise     1722
sadness      1583
fear          689
anger         591
disgust        93
Name: sentiment, dtype: int64

In [16]:
df=pd.read_csv('march_sentiment.csv')
df['sentiment'].value_counts()

neutral     32181
joy          5797
surprise     1667
sadness      1184
fear         1020
anger         285
disgust       184
Name: sentiment, dtype: int64

In [17]:
df2=pd.read_csv('may_sentiment.csv')
df2['sentiment'].value_counts()

neutral     33325
joy          6444
sadness      1993
surprise     1991
fear          546
anger         476
disgust       141
Name: sentiment, dtype: int64

In [22]:
df3=pd.read_csv('june_sentiment.csv')
df3['sentiment'].value_counts()

neutral     35503
joy          5846
surprise     1852
sadness      1556
fear          605
anger         391
disgust       134
Name: sentiment, dtype: int64